<a href="https://colab.research.google.com/github/shengpu-tang/CS334-S25/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 - Predicting Survival of ICU Patients

In [ ]:
#@title Run this code block to download and unzip data
#@markdown - If you already have the data in your workspace, you can skip this step.
#@markdown - If you want to reset your workspace, use menu "Runtime - Disconnect and delete runtime".
#@markdown - To save your work in this notebok, use menu "File - Save a copy in Drive..."

!rm -rf config.yaml data.zip data
!wget -q "https://raw.githubusercontent.com/shengpu-tang/CS334-F24/main/HW3/config.yaml"
!wget -q "https://raw.githubusercontent.com/shengpu-tang/CS334-F24/main/HW3/data.zip"
!unzip -qq data.zip

In [ ]:
# @title HW3 - helper.py
# @markdown This block contains the content of the helper file so you don't need to download it separately.

import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split

import yaml
config = yaml.load(open('config.yaml'), Loader=yaml.FullLoader)

def get_train_test_split():
    """
    This function performs the following steps:
    - Reads in the data from data/labels.csv and data/files/*.csv (keep only the first 2,500 examples)
    - Generates a feature vector for each example
    - Aggregates feature vectors into a feature matrix (features are sorted alphabetically by name)
    - Performs imputation and normalization with respect to the population

    After all these steps, it splits the data into 80% train and 20% test.

    The binary labels take two values:
        -1: survivor
        +1: died in hospital

    Returns the features and labesl for train and test sets, followed by the names of features.
    """
    df_labels = pd.read_csv('data/labels.csv')
    df_labels = df_labels[:2500]
    IDs = df_labels['RecordID'][:2500]
    raw_data = {}
    for i in tqdm(IDs, desc='Loading files from disk'):
        raw_data[i] = pd.read_csv('data/files/{}.csv'.format(i))

    features = Parallel(n_jobs=16)(delayed(generate_feature_vector)(df) for _, df in tqdm(raw_data.items(), desc='Generating feature vectors'))
    df_features = pd.DataFrame(features).sort_index(axis=1)
    feature_names = df_features.columns.tolist()
    X, y = df_features.values, df_labels['In-hospital_death'].values
    X = impute_missing_values(X)
    X = normalize_feature_matrix(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=3)
    return X_train, y_train, X_test, y_test, feature_names


def get_classifier(penalty='l2', C=1.0, class_weight=None):
    """
    Returns a logistic regression classifier based on the given
    penalty function, regularization parameter C, and class weights.
    """
    if penalty == 'l2':
        return LogisticRegression(penalty=penalty, C=C, class_weight=class_weight, solver='lbfgs', max_iter=1000)
    elif penalty == 'l1':
        return LogisticRegression(penalty=penalty, C=C, class_weight=class_weight, solver='saga', max_iter=5000)
    else:
        raise ValueError('Error: unsupported configuration')


def make_challenge_submission(y_label, y_score):
    """
    Takes in `y_label` and `y_score`, which are two list-like objects that contain
    both the binary predictions and raw scores from your classifier.
    Outputs the prediction to challenge.csv.

    Please make sure that you do not change the order of the test examples in the heldout set
    since we will use this file to evaluate your classifier.
    """
    print('Saving challenge output...')
    pd.DataFrame({'label': y_label.astype(int), 'risk_score': y_score}).to_csv('challenge.csv', index=False)
    print('challenge.csv saved')
    return


def test_challenge_output():
    import csv
    with open('challenge.csv', newline='') as csvfile:
        filereader = csv.reader(csvfile)
        i = 0
        for row in filereader:
            if i == 0:
                if row[0] != 'label':
                    print('INVALID COLUMN NAME: column name is not label.')
            else:
                rating = int(row[0])
                if rating != -1 and rating != 0 and rating != 1:
                    print('INVALID VALUE: values need to be -1, 0, or 1.')
            i += 1
        if i != 2001:
            print('INVALID NUMBER OF ROWS: number of rows is not 2001.')
        print('SUCCESS: csv file is valid.')
    return

## Import packages

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, exceptions
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=exceptions.UndefinedMetricWarning)

## 1. Feature Extraction

In [ ]:

def generate_feature_vector(df):
    """
    Reads a dataframe containing all measurements for a single patient
    within the first 48 hours of the ICU admission, and convert it into
    a feature vector.

    Input:
        df: pd.Dataframe, with columns [Time, Variable, Value]

    Returns:
        a python dictionary of format {feature_name: feature_value}
        for example, {'Age': 32, 'Gender': 0, 'mean_HR': 84, ...}
    """
    static_variables = config['static']
    timeseries_variables = config['timeseries']
    feature_dict = {}

    # TODO: Implement this function
    ???

    return feature_dict


def impute_missing_values(X):
    """
    For each feature column, impute missing values  (np.nan) with the
    population mean for that feature.

    Input:
        X: np.array, shape (N, d). X could contain missing values
    Returns:
        X: np.array, shape (N, d). X does not contain any missing values
    """
    # TODO: Implement this function
    ???
    return X


def normalize_feature_matrix(X):
    """
    For each feature column, normalize all values to range [0, 1].

    Input:
        X: np.array, shape (N, d).
    Returns:
        X: np.array, shape (N, d). Values are normalized per column.
    """
    # TODO: Implement this function
    ???
    return X


In [ ]:
def q1(X, feature_names):
    """
    Given a feature matrix X, prints d, the number of features in the feature vector,
    and prints the average feature vector along with its corresponing feature name.
    """
    ##################################################################
    print("--------------------------------------------")
    print("Question 1(d): reporting dataset statistics:")
    print("\t", "d:", X.shape[1])
    print("\t", "Average feature vector:")
    print(pd.DataFrame({"Feature Name": feature_names, "Mean value": X.mean(axis=0)}))

In [ ]:
# Read data
# NOTE: READING IN THE DATA WILL NOT WORK UNTIL YOU HAVE FINISHED
#       IMPLEMENTING generate_feature_vector, fill_missing_values AND normalize_feature_matrix
X_train, y_train, X_test, y_test, feature_names = get_train_test_split()

In [ ]:
q1(X_train, feature_names)

## 2. Hyperparameter and Model Selection

In [ ]:
def performance(clf, X, y_true, metric='accuracy'):
    """
    Calculates the performance metric as evaluated on the true labels
    y_true versus the predicted scores from clf and X.
    Input:
        clf: an instance of sklearn estimator
        X : (N,d) np.array containing features
        y_true: (N,) np.array containing true labels
        metric: string specifying the performance metric (default='accuracy'
                 other options: 'precision', 'sensitivity', 'specificity',
                'f1-score', 'auroc', and 'auprc')
    Returns:
        the performance measure as a float
    """
    # TODO: Implement this function
    ???


def cv_performance(clf, X, y, k=5, metric='accuracy'):
    """
    Splits the data X and the labels y into k folds.
    Then, for each fold i in 1...k,
        Train a classifier on all the data except the i-th fold, and test on the i-th fold.
        Calculate the performance of the classifier and save the result.
    In the end, return the average performance across the k folds.
    Input:
        clf: an instance of sklearn estimator
        X: (N,d) array of feature vectors, where n is the number of examples
           and d is the number of features
        y: (N,) array of binary labels {1,-1}
        k: an int specifying the number of folds (default=5)
        metric: string specifying the performance metric (default='accuracy'
             other options: 'precision', 'sensitivity', 'specificity',
                'f1-score', 'auroc', and 'auprc')
    Returns:
        average cross-validation performance across the k folds as a float
    """
    # TODO: Implement this function
    skf = StratifiedKFold(n_splits=k)
    scores = []
    # For each split in the k folds...
    for train, val in skf.split(X,y):
        # Split the data into training and validation sets...
        X_train, y_train, X_val, y_val = ???
        # Fit the data to the training data...
        ???
        # And test on the ith fold.
        score = ???
        scores.append(score)
    # Return the average performance across all fold splits.
    return np.array(scores).mean()


def select_C(X, y, C_range=[], penalty='l2', k=5, metric='accuracy'):
    """
    Sweeps different C hyperparameters of a logistic regression classifier,
    calculates the k-fold CV performance for each setting on dataset (X, y),
    and return the best C.
    Input:
        X: (N,d) array of feature vectors, where N is the number of examples
            and d is the number of features
        y: (N,) array of binary labels {1,-1}
        k: int specifying the number of folds for cross-validation (default=5)
        metric: string specifying the performance metric (default='accuracy',
             other options: 'precision', 'sensitivity', 'specificity',
                'f1-score', 'auroc', and 'auprc')
        penalty: whether to use 'l1' or 'l2' regularization (default='l2')
        C_range: a list of hyperparameter C values to be searched over
    Returns:
        the C value for a logistic regression classifier that maximizes
        the average 5-fold CV performance.
    """
    print("{}-regularized Logistic Regression "
          "Hyperparameter Selection based on {}:".format(penalty.upper(), metric))
    scores = []
    # Iterate over all of the given C range...
    for C in C_range:
        # Calculate the average performance on k-fold cross-validation
        clf = get_classifier(penalty=penalty, C=C)
        score = cv_performance(clf, X, y, k, metric)
        print("C: {:.6f} \t score: {:.4f}".format(C, score))
        scores.append((C, score))
    # Return the C value with the maximum score
    maxval = max(scores, key=lambda x: x[1])
    return maxval[0]


def plot_coefficients(X, y, penalty, C_range):
    """
    Takes as input the training data X and labels y and plots the L0-norm
    (number of nonzero elements) of the coefficients learned by a classifier
    as a function of the C-values of the classifier.
    """
    print("Plotting the number of nonzero entries of the parameter vector as a function of C")
    norm0 = []

    # TODO: Implement this function
    ???

    # This code will plot your L0-norm as a function of C
    plt.plot(C_range, norm0)
    plt.axhline(y=X.shape[1], color='gray', linestyle=':')
    plt.xscale('log')
    plt.legend(['L0-norm'])
    plt.xlabel("Value of C")
    plt.ylabel("L0-norm of theta")
    plt.ylim(-2,50)
    plt.title('L0-norm of θ vs C, {}-penalized logistic regression'.format(penalty.upper()))
    plt.savefig('l0-norm_vs_C__'+penalty+'-penalty.png', dpi=200)
    plt.close()

    print('Plot saved')

In [ ]:
def q2(X_train, y_train, X_test, y_test, metric_list, feature_names):
    """
    This function should contain all the code you implement to complete part 2
    """
    print("================= Part 2 ===================")

    C_range = np.logspace(-3, 3, 7)

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.1(c): Logistic Regression with L2-penalty, grid search, all metrics")
    for metric in metric_list:
        best_C = select_C(X_train, y_train, C_range, 'l2', 5, metric)
        print("Best C: %.6f" % best_C)

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.1(d): Test Performance of L2-reg logistic regression with best C")
    best_C  = ???
    ??? # Fit the classifier with the best C
    for metric in metric_list:
        test_perf = performance(clf, X_test, y_test, metric)
        print("C = " + str(best_C) + " Test Performance on metric " + metric + ": %.4f" % test_perf)

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.1(e): Plot L0-norm of theta coefficients vs. C, l2 penalty")
    plot_coefficients(X_train, y_train, 'l2', C_range)

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.1(f): Displaying the most positive and negative coefficients and features")
    best_C = ???
    print('Positive coefficients...')
    print('Negative coefficients...')

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.2(a): Logistic Regression with L1-penalty, grid search, AUROC")
    best_C = ???
    test_performance = ???

    ##################################################################
    print("--------------------------------------------")
    print("Question 2.2(b): Plot the weights of C vs. L0-norm of theta, l1 penalty")
    plot_coefficients(X_train, y_train, 'l1', C_range)

In [ ]:
metric_list = ["accuracy", "precision", "sensitivity", "specificity", "f1_score", "auroc", "auprc"]

In [ ]:
q2(X_train, y_train, X_test, y_test, metric_list, feature_names)

## 3. Challenge

In [ ]:
def generate_feature_vector_challenge(df):
    return generate_feature_vector(df)

def impute_missing_values_challenge(X):
    return impute_missing_values(X)

def normalize_feature_matrix_challenge(X):
    return normalize_feature_matrix(X)

In [ ]:
def get_challenge_data():
    """
    This function is similar to helper.get_train_test_split, except that:
    - It reads in all 10,000 training examples
    - It does not return labels for the 2,000 examples in the heldout test set
    You should replace your preprocessing functions (generate_feature_vector,
    impute_missing_values, normalize_feature_matrix) with updated versions for the challenge
    """
    df_labels = pd.read_csv('data/labels.csv')
    df_labels = df_labels
    IDs = df_labels['RecordID']
    raw_data = {}
    for i in tqdm(IDs, desc='Loading files from disk'):
        raw_data[i] = pd.read_csv('data/files/{}.csv'.format(i))

    features = Parallel(n_jobs=16)(delayed(generate_feature_vector_challenge)(df) for _, df in tqdm(raw_data.items(), desc='Generating feature vectors'))
    df_features = pd.DataFrame(features)
    feature_names = df_features.columns.tolist()
    X, y = df_features.values, df_labels['30-day_mortality'].values
    X = impute_missing_values_challenge(X)
    X = normalize_feature_matrix_challenge(X)
    return X[:10000], y[:10000], X[10000:], feature_names

In [ ]:
def run_challenge(X_challenge, y_challenge, X_heldout):
    # TODO:
    # Read challenge data
    # Train a linear classifier and apply to heldout dataset features
    # Use generate_challenge_labels to print the predicted labels
    print("================= Part 3 ===================")
    print("Part 3: Challenge")
    clf = LogisticRegression() ### TODO: define your classifier with appropriate hyperparameters
    clf.fit(X_challenge, y_challenge)
    y_score = clf.decision_function(X_heldout)
    y_label = clf.predict(X_heldout)
    make_challenge_submission(y_label, y_score)

In [ ]:
# Read challenge data
X_challenge, y_challenge, X_heldout, feature_names = get_challenge_data()

In [ ]:
# TODO: Question 3: Apply a classifier to heldout features, and then use
#       generate_challenge_labels to print the predicted labels
run_challenge(X_challenge, y_challenge, X_heldout)
test_challenge_output()